In [9]:
import numpy as np
import urllib.request
from LUSI import LUSI_exact, LUSI_approx, LUSI_SVM
from utils import load_data
from predicates import f1, f2, f3
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.utils.estimator_checks import check_estimator
from sklearn.svm import SVC

In [2]:
# Download MNIST dataset "mnist.npz"
url = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
urllib.request.urlretrieve(url, "mnist.npz")

('mnist.npz', <http.client.HTTPMessage at 0x119dc0e20>)

In [2]:
X_train, y_train, X_test, y_test = load_data("mnist.npz")
print("X_train", X_train.shape, "y_train", y_train.shape)

X_train (2000, 784) y_train (2000,)


## LUSI_exact

## No predicates

In [19]:
def model_eval(class_name, predicate, hyper_params, X_train, y_train, X_test, y_test):
    # creating a KFold object with 5 splits 
    folds = KFold(n_splits = 5, shuffle = True, random_state = 10)

    # specify range of hyperparameters
    # Set the parameters by cross-validation
    
    if predicate is None:
        predicate_size = 0
        X_train_ext = X_train
    else:
        Phi = predicate(X_train)
        #norm = np.sqrt(np.sum(Phi**2, axis=0, keepdims=True)) #normalization
        #norm[norm < 0.0001] = 1
        #Phi = Phi / norm
        predicate_size = Phi.shape[1]
        X_train_ext = np.concatenate((X_train, Phi), axis = 1)
    
    model = class_name(predicate_size = predicate_size)
    # set up GridSearchCV()
    model_cv = GridSearchCV(estimator = model, 
                            param_grid = hyper_params, 
                            scoring= 'accuracy', 
                            cv = folds, 
                            verbose = 1,
                            return_train_score=True,
                            n_jobs=10)      

    # Cross validation
    %time model_cv.fit(X_train_ext, y_train)
    best_score = model_cv.best_score_
    best_hyperparams = model_cv.best_params_

    print("The best test score is {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))
    # Fit the final model with optimal hyperparameters
    model.set_params(**best_hyperparams)
    model.fit(X_train_ext, y_train)
    y_train_pred = model.predict(X_train_ext)
    y_test_pred = model.predict(np.concatenate( (X_test, np.zeros((X_test.shape[0], predicate_size))), axis=1))

    accuracy_train = np.mean(y_train_pred==y_train)
    accuracy_test = np.mean(y_test_pred==y_test)
    print("Train accuracy = ", accuracy_train)
    print("Test accuracy = ", accuracy_test)
    

In [20]:
hyper_params = [ {'tau': [1e-1, 1e-2, 1e-3, 1e-4],
                 'gamma': ['scale', 0.01, 0.1, 1],
                 'V_mtx': ['add', 'kde1', 'I']}]

model_eval(class_name=LUSI_exact, 
           predicate=None, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
CPU times: user 12.1 s, sys: 1.72 s, total: 13.8 s
Wall time: 2min 28s
The best test score is 0.9380000000000001 corresponding to hyperparameters {'V_mtx': 'add', 'gamma': 'scale', 'tau': 0.0001}
Train accuracy =  1.0
Test accuracy =  0.946


## f1: constant predicate

In [21]:
hyper_params = [ {'tau': [1e-1, 1e-2, 1e-3, 1e-4],
                     'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_exact, 
           predicate=f1, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
CPU times: user 13.7 s, sys: 1.77 s, total: 15.5 s
Wall time: 1min 5s
The best test score is 0.9380000000000001 corresponding to hyperparameters {'gamma': 'scale', 'tau': 0.0001}
Train accuracy =  1.0
Test accuracy =  0.946


## f2: identity predicate

In [26]:
hyper_params = [ {'tau': [1e-1, 1e-2, 1e-3, 1e-4],
                     'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_exact, 
           predicate=f2, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
CPU times: user 1h 52min 40s, sys: 4min 48s, total: 1h 57min 28s
Wall time: 2h 45min 35s
The best test score is 0.9380000000000001 corresponding to hyperparameters {'gamma': 'scale', 'tau': 0.0001}
Train accuracy =  1.0
Test accuracy =  0.946


## f3: tangent distance predicate

In [18]:
hyper_params = [ {'tau': [1e-1, 1e-2, 1e-3, 1e-4],
                     'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_exact, 
           predicate=f3, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
CPU times: user 13.6 s, sys: 2.32 s, total: 15.9 s
Wall time: 1min 11s
The best test score is 0.9380000000000001 corresponding to hyperparameters {'gamma': 'scale', 'tau': 0.0001}
Train accuracy =  1.0
Test accuracy =  0.9459


# LUSI_approx

## No predicate

In [24]:
hyper_params = [ {'tau': [0.01, 0.1, 0.5, 0.9, 0.99],
                  'lamda': [0.1, 0.01, 0.001], 
                  'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_approx, 
           predicate=None, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 19.9 s, sys: 2.3 s, total: 22.3 s
Wall time: 4min 22s
The best test score is 0.9349999999999999 corresponding to hyperparameters {'gamma': 'scale', 'lamda': 0.001, 'tau': 0.01}
Train accuracy =  1.0
Test accuracy =  0.9439


## Predicate f1

In [25]:
hyper_params = [ {'tau': [0.01, 0.1, 0.5, 0.9, 0.99],
                  'lamda': [0.1, 0.01, 0.001], 
                  'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_approx, 
           predicate=f1, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 21.8 s, sys: 2.35 s, total: 24.2 s
Wall time: 4min 59s
The best test score is 0.9349999999999999 corresponding to hyperparameters {'gamma': 'scale', 'lamda': 0.001, 'tau': 0.9}
Train accuracy =  1.0
Test accuracy =  0.9438


## Predicate f2

In [23]:
hyper_params = [ {'tau': [0.01, 0.1, 0.5, 0.9, 0.99],
                  'lamda': [0.1, 0.01, 0.001], 
                  'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_approx, 
           predicate=f2, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 21.9 s, sys: 2.26 s, total: 24.1 s
Wall time: 5min 3s
The best test score is 0.9349999999999999 corresponding to hyperparameters {'gamma': 'scale', 'lamda': 0.001, 'tau': 0.9}
Train accuracy =  1.0
Test accuracy =  0.9438


## Predicate f3

In [22]:
hyper_params = [ {'tau': [0.01, 0.1, 0.5, 0.9, 0.99],
                  'lamda': [0.1, 0.01, 0.001], 
                  'gamma': ['scale', 0.01, 0.1, 1]}]

model_eval(class_name=LUSI_approx, 
           predicate=f3, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 22 s, sys: 2.22 s, total: 24.3 s
Wall time: 4min 58s
The best test score is 0.9355 corresponding to hyperparameters {'gamma': 'scale', 'lamda': 0.001, 'tau': 0.9}
Train accuracy =  1.0
Test accuracy =  0.9435


# LUSI_SVM

## No predicate

In [6]:
hyper_params = [ {'C': [0.1, 1],
                  'epsilon_star': [0.0001, 0.01], 
                  'tau': [0.1, 0.5, 0.9]}]

model_eval(class_name=LUSI_SVM, 
           predicate=None, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-

CPU times: user 15min 52s, sys: 27.5 s, total: 16min 19s
Wall time: 59min 54s
The best test score is 0.9279999999999999 corresponding to hyperparameters {'C': 0.1, 'epsilon_star': 0.01, 'tau': 0.1}


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


Train accuracy =  1.0
Test accuracy =  0.9375


## Predicate f1

In [7]:
hyper_params = [ {'C': [0.1, 1],
                  'epsilon_star': [0.0001, 0.01], 
                  'tau': [0.1, 0.5, 0.9]}]

model_eval(class_name=LUSI_SVM, 
           predicate=f1, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-

CPU times: user 7min 11s, sys: 27.8 s, total: 7min 39s
Wall time: 37min 45s
The best test score is 0.8935000000000001 corresponding to hyperparameters {'C': 0.1, 'epsilon_star': 0.01, 'tau': 0.1}


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


Train accuracy =  1.0
Test accuracy =  0.8926


## Predicate f3

In [8]:
hyper_params = [ {'C': [0.1, 1],
                  'epsilon_star': [0.0001, 0.01], 
                  'tau': [0.1, 0.5, 0.9]}]

model_eval(class_name=LUSI_SVM, 
           predicate=f3, 
           hyper_params=hyper_params, 
           X_train=X_train, 
           y_train=y_train, 
           X_test=X_test, 
           y_test=y_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-

CPU times: user 12min 29s, sys: 24 s, total: 12min 53s
Wall time: 56min 58s
The best test score is 0.8975 corresponding to hyperparameters {'C': 1, 'epsilon_star': 0.01, 'tau': 0.1}


/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/zjgong/miniconda3/envs/ML/lib/python3.8/site-packages/cvxpy/problems/problem.py:1296: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Train accuracy =  1.0
Test accuracy =  0.8972


# SVM

In [16]:
hyper_params = [ {'C': [0.1, 0.01, 0.001], 
                 'gamma': ['auto', 'scale', 1, 0.1, 0.01, 0.001]}]


folds = KFold(n_splits = 5, shuffle = True, random_state = 10)
model = SVC(kernel='rbf')
# set up GridSearchCV()
model_cv = GridSearchCV(estimator = model, 
                        param_grid = hyper_params, 
                        scoring= 'accuracy', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True,
                        n_jobs=10)      

# Cross validation
%time model_cv.fit(X_train, y_train)
best_score = model_cv.best_score_
best_hyperparams = model_cv.best_params_

print("The best test score is {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))
# Fit the final model with optimal hyperparameters
model.set_params(**best_hyperparams)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

accuracy_train = np.mean(y_train_pred==y_train)
accuracy_test = np.mean(y_test_pred==y_test)
print("Train accuracy = ", accuracy_train)
print("Test accuracy = ", accuracy_test)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
CPU times: user 2.13 s, sys: 31.3 ms, total: 2.16 s
Wall time: 44.1 s
The best test score is 0.8560000000000001 corresponding to hyperparameters {'C': 0.1, 'gamma': 'scale'}
Train accuracy =  0.896
Test accuracy =  0.8675
